## Example `stata_kernel` Jupyter notebook

This Jupyter notebook is an example of how you can use Stata in the Jupyter ecosystem using `stata_kernel`.

Full documentation, including how to install, is available at https://kylebarron.dev/stata_kernel/.

## Overview

The Jupyter Notebook is a file format that permits interactive coding with text, code, and results in a single document. You can share a notebook file (with extension `.ipynb`), and results will be viewable without running the code, but as long as the recipient also has Jupyter installed, he or she can edit and re-run the code cells.

Jupyter itself is language agnostic, i.e. it permits writing code in any language. This document uses Stata code, but you can also code in Jupyter using Python, [R](https://irkernel.github.io/), [Julia](https://github.com/JuliaLang/IJulia.jl), [Matlab](https://github.com/calysto/matlab_kernel), and [SAS](https://github.com/sassoftware/sas_kernel). 

## Running code

In contrast to [IPyStata](https://github.com/TiesdeKok/ipystata), no special commands are needed. Just write code as you would normally in Stata.

Let's make sure that the connection with Stata is working properly.

In [1]:
display "Hello, world!"

SyntaxError: invalid syntax (<ipython-input-1-635beafc47b6>, line 1)

In [ ]:
sysuse auto,clear

In [ ]:
scatter price mpg

In [ ]:
use "https://stats.idre.ucla.edu/stat/stata/notes/hsb2", clear
scatter read math, title("Reading score vs Math score")
scatter math science, title("Math score vs Science score")

In [ ]:
version

You can run a cell by pressing <kbd>Ctrl</kbd>+<kbd>Enter</kbd> or <kbd>Shift</kbd>+<kbd>Enter</kbd>. If a number appears in the brackets to the left of the input cell, that means that the code was successfully run (sometimes a cell doesn't produce any output).

If you don't see `Hello, world!` as output, check out the [troubleshooting tips](https://kylebarron.dev/stata_kernel/using_stata_kernel/troubleshooting/).

Let's load the included `auto` dataset.

In [ ]:
reg price mpg

Now the `auto` dataset is in memory.

### Basic descriptive statistics

Nearly all commands that work in Stata work through Jupyter as well. A couple commands that depend on the Graphical User Interface, such as `browse` and `edit`, only work on Windows.

In [ ]:
tabulate foreign headroom

## Graphs

No special syntax is needed to generate graphs. Just write commands like you're used to.
The display order of graphs will always be the same as the order in the code.

In [ ]:
sysuse auto,clear

In [ ]:
scatter price mpg

If you don't want to display a graph, just prefix the command with [`quietly`](https://www.stata.com/help.cgi?quietly).

In [ ]:
reg price mpg

## Comments in code

`stata_kernel` lets you use _any_ format of comments, including `//`, `///`, `*`, and `/*`-`*/`, even in an interactive console environment where the Stata command line normally wouldn't accept them.

In [ ]:
display "displayed"
// display "comment"

In [ ]:
display "line continuation " /// comment
    "comment"

In [ ]:
* display "not displayed"

In [ ]:
display "displayed1"
/*
display "displayed2"
*/
display "displayed3"

## Autocompletion

`stata_kernel` provides autocompletion for locals, globals, variables, scalars, and matrices based on the contents in memory. It also suggests file paths to load or save files. Press <kbd>Tab</kbd> while typing to activate it.

![](https://raw.githubusercontent.com/kylebarron/stata_kernel/master/docs/src/img/jupyterlab_autocompletion.png)

## Magics

[_Magics_](https://kylebarron.dev/stata_kernel/using_stata_kernel/magics/) are special commands that `stata_kernel` provides to give extra functionality, especially regarding the connection with Jupyter. 

These commands all start with `%`. You can run `%help magics` or [go here](https://kylebarron.dev/stata_kernel/using_stata_kernel/magics/) to see a list of available magics. You can also run `%magic_name --help` to see the help for any given magic.

In order to prevent confusion, these commands **must** occur at the beginning of a cell.

### `%head`, `%browse`, `%tail`

**`%head`**, **`%browse`**, and **`%tail`** show a well-formatted portion of the dataset in memory.

In [ ]:
%head 5

### `%help`
**`%help`** shows the help menu for a given command. The links in this help file are clickable, just like the official Stata documentation. (This command requires internet access.)

In [ ]:
%help summarize

### `%locals`, `%globals`

**`%locals`** and **`%globals`** display the local or global macros in the current environment.

In [ ]:
local local1 "foo"
local local2 "bar"
local abcd "foo bar"

In [ ]:
%help reg

In [ ]:
%locals

In [ ]:
%locals loc

In [ ]:
%globals

### `%html`, `%latex`

**`%html`** and **`%latex`** attempt to display either type of _output_ (not user input). This could be used, for example, with `estout` to display several regression results side-by-side.

**Note:** Jupyter can display a math subset of LaTeX but doesn't support tables. _However_, it's really easy to export a Jupyter Notebook file to PDF through LaTeX (see File > Export Notebook As > Export Notebook to PDF). In this PDF export, LaTeX tables _will_ be properly displayed.

In [ ]:
cap ssc install estout
sysuse auto, clear
eststo clear
eststo: qui regress price mpg rep78
eststo: qui regress price mpg rep78 gear_ratio trunk
eststo: qui regress price mpg rep78 gear_ratio trunk weight displacement

In [ ]:
%html
esttab, label title("Regression Table") html

### `%show_gui`, `%hide_gui`

On Windows, **`%show_gui`** and **`%hide_gui`** show and hide the traditional Stata Graphical User Interface window. These magics do not work on macOS or Linux because those platforms communicate with Stata in a different manner.

## `;`-delimited commands

Often with long commands, such as graphs, using [`#delimit ;`](https://www.stata.com/help.cgi?delimit) helps prevent very long lines and helps to keep code more readable. This is supported in `stata_kernel`, despite it not being allowed in the normal Stata command-line environment.

In [ ]:
sysuse auto, clear

In [ ]:
#delimit ;
display "Hello, world!";

It's important to note that the `;`-delimiter mode persists across cells. `stata_kernel` will expect cells to include `;` for each command, and will raise an error if `;` is missing.

In [ ]:
display "Hello, world!"

You can check the current delimiter with the **`%delimit`** magic.

In [ ]:
%delimit

You can switch back to normal line-break delimited commands (i.e. where `;` is unnecessary) with `#delimit cr`.

In [ ]:
#delimit cr

## Using Mata

You can start an interactive Mata session by typing `mata`. This persists across cells; cells will continue being Mata cells until you run `end` to exit the mata session.

You can run the **`%status`** magic to check if you're in Mata or Stata mode.

In [ ]:
sysuse auto, clear

In [ ]:
mata

In [ ]:
%status

In [ ]:
y    = st_data(., "price")
X    = st_data(., "mpg trunk")
n    = rows(X)
X    = X,J(n,1,1)
XpX  = quadcross(X, X)
XpXi = invsym(XpX)
b    = XpXi*quadcross(X, y)

In [ ]:
b'

In [ ]:
end

In [ ]:
%status